#**BBQ-mIS**
Graph coloring optimization on neutral atoms quantum machines.
* clone repository (codes building blocks and graphs dataset)
* install needed libraries
* investigate BB behaviour by changing 
  * exploration policy, i.e. queue_strategy (https://pybnb.readthedocs.io/en/stable/index.html)
  * feasible coloring computation
  * maximum number of explored nodes

In [1]:
!git clone https://github.com/LINKS-Foundation-CPE/Master-QCC-2022-2023.git

Cloning into 'Master-QCC-2022-2023'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 102 (delta 47), reused 89 (delta 37), pack-reused 0
Receiving objects: 100% (102/102), 78.11 KiB | 15.62 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [1]:
!pip install pybnb
!pip install mpi4py
!pip install pulser==0.17.14
!pip install networkx==2.8.7
!pip install matplotlib==3.5.1
!pip install matplotlib-inline

ERROR: Could not find a version that satisfies the requirement pulser==0.17.14 (from versions: 0.0.1a1, 0.1.0, 0.1.1, 0.1.2, 0.2.0, 0.2.1, 0.3.0, 0.3.1, 0.3.2, 0.4.0, 0.4.1, 0.4.2, 0.5.0, 0.5.1, 0.6.0, 0.7.0, 0.7.1, 0.7.2, 0.7.3, 0.8.0, 0.8.1, 0.9.0, 0.9.1, 0.9.2, 0.9.3, 0.10.0, 0.11.0, 0.11.1, 0.12.0, 0.13.0, 0.13.1, 0.13.2, 0.13.3, 0.14.0, 0.14.1, 0.15.0, 0.15.1, 0.15.2, 0.15.3, 0.16.0, 0.16.1, 0.16.2, 0.17.0, 0.17.1, 0.17.2, 0.17.3, 0.17.4)
ERROR: No matching distribution found for pulser==0.17.14


In [ ]:
%cd /content/Master-QCC-2022-2023/

In [2]:
%matplotlib inline

In [3]:
def compute_obj(G, colors_used, previous_coloring):
    if len(G.edges())>0:
        obj = colors_used+len(G.nodes())
        for greedy_color_node in G.nodes():
            colors_used = colors_used + 1
            previous_coloring[greedy_color_node] = colors_used
    else:
        for node in G.nodes():
            previous_coloring[node]=colors_used+1
        obj = colors_used+1
    return obj, previous_coloring

In [4]:
import pybnb
import numpy as np
from src.graph_coloring.BB_utils import *
from src.graph_coloring.MIS import PulserMISSolver
from src.graph_coloring.feasibility_utils import is_mIS

class BBQ_mIS(pybnb.Problem):
    
    def __init__(self, G, colors_used=0):
        self.G = G 
        self.orig_G = G.copy()     
        self.x = np.zeros((len(G.nodes),1))
        _, LB, _ = compute_LB(self.x, self.G)
        self.colors_used = colors_used
        self.edges = len(G.edges())
        self.fingerprints=set([])        
        self.lower_bound=colors_used+LB
        coloring =  dict.fromkeys(G.nodes(), -1)
        obj, coloring = compute_obj(G, colors_used, coloring)
        self.obj = obj
        self.coloring = coloring
        self.child_story={0:-1}
        self.hist_list = []
        self.solutions = None

    def sense(self):
        return pybnb.minimize

    def objective(self):             
        return self.obj

    def bound(self):
        # lower bound on the objective function
        return self.lower_bound

    def save_state(self, node, x=None, colors_used=None, lower_bound=None, H=None, obj=None, coloring=None, child_story=None, solutions = None, hist_list=None):
        if x is None:
            # root node initialization
            node.state = (self.x, self.colors_used, self.G, self.edges, self.coloring, self.child_story, self.solutions, self.hist_list)
        else:
            num_edges = len(H.edges())
            node.state = (x, colors_used, H, num_edges, coloring, child_story, solutions, hist_list)
            node.objective = obj
            node.bound = lower_bound
            UB = compute_UB(H)
            node.queue_priority = -UB*num_edges
                          
            
    def load_state(self, node):
        (self.x, self.colors_used, self.G, self.edges, self.coloring, self.child_story, self.solutions, self.hist_list) = node.state
        self.obj = node.objective
        self.lower_bound = node.bound

    def branch(self):
        pulser_MIS_solver = PulserMISSolver(self.G)
        pulser_sol = pulser_MIS_solver.solve_Pulser()
        hist_list=self.hist_list.copy()
        hist_list.append(pulser_sol)
        num_colors_child = self.colors_used+1
        solutions = []  
        for sol in pulser_sol:
            solutions.append(np.array(list(sol[0]), dtype=int))
        num_sol = len(solutions)
        child_num=0                    
        for sol in range(num_sol): 
            child_story=self.child_story.copy()
            child_story[num_colors_child]=child_num+1
            coloring_dict = self.coloring.copy() 
            x = solutions[sol]      
            if is_mIS(x, self.G):                                
                H, LB, MIS_set = compute_LB(x, self.G)
                for graph_node in MIS_set:
                    coloring_dict[graph_node]=num_colors_child                
                fp = fingerprint(H.nodes())                
                if fp not in self.fingerprints and len(H.nodes())>0:
                    child_num+=1
                    # avoid symmetries in BB
                    child = pybnb.Node()
                    obj, coloring_dict = compute_obj(H, num_colors_child, coloring_dict)
                    child_bound = num_colors_child+LB
                    self.save_state(child, x, num_colors_child, child_bound, H, obj, coloring_dict, child_story, pulser_sol, hist_list)
                    self.fingerprints.add(fp)
                    yield child

NameError: name 'RydbergEOM' is not defined

In [5]:
import networkx as nx
from src.graph_coloring.BB_utils import *
import warnings

file_path="graphs/{}G_{}ud.gpickle".format(7, 2)
G = nx.read_gpickle(file_path)
max_nodes=20
problem = BBQ_mIS(G)
solver = pybnb.Solver()
results = solver.solve(problem, queue_strategy='custom', node_limit=max_nodes)
best_coloring = results.best_node.state[4]
child_story = results.best_node.state[5]
plot_sol(G, best_coloring, results.objective)    
print_BB_history(child_story)
# print the MISs for the best solution
print_mIS(best_coloring)
print('Coloring story {}'.format(child_story))
print('BBQ-mIS found coloring with {} colors'.format(results.objective))


Using non-default solver options:
 - node_limit: 20 (default: None)
 - queue_strategy: custom (default: bound)

Starting branch & bound solve:
 - dispatcher pid: 14596 (LAPTOP-SIDQI55K)
 - worker processes: 1
--------------------------------------------------------------------------------------------------------------------------
         Nodes        |                      Objective Bounds                       |              Work              
      Expl    Unexpl  |      Incumbent           Bound    Rel. Gap         Abs. Gap | Time (s)  Nodes/Sec Imbalance   Idle
         0         1  |            inf            -inf         inf%             inf |      0.0       0.00     0.00%      0


Exception caught: No channel rydberg_global in the device.
Attempting to shut down, but this may hang.


ValueError: No channel rydberg_global in the device.